# Scoring and Evaluation AutoAI Model with Test Data

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## AutoAI deployment model Evaluation

In [ ]:
import pandas as pd
import numpy as np
from itertools import groupby 
from collections import OrderedDict
import json

# READING AND WRITING PROJECT ASSETS
import project_lib
project = project_lib.Project()

df = pd.read_csv(project.get_file('df_validation.csv'), dtype=str)
df.head()

In [ ]:
results_df = df[["loan_id", "default"]]
results_df["default"].replace({"True": True, "False": False}, inplace=True)

In [ ]:
df.columns

In [ ]:
#Preprocessing similar to train data, uncomment these lines if you had deactivated the column selector flag in the model pipeline
#df.drop(['loan_id', 'prediction_date', 'default', 'date_of_default', 'date_of_first_payment', 
#         "date_of_last_payment","default_12mth", 'branch_id']
#        ,axis=1, inplace=True)

In [ ]:
df.drop(['default']
        ,axis=1, inplace=True)

df.fillna(0, inplace=True)
df = df.astype(str)

In [ ]:
#Creating payload data
payload_array_of_input_fields = np.array(df.columns)
print(payload_array_of_input_fields)

array_of_values_to_be_scored = []
for i in range((df.shape[0])): 
    array_of_values_to_be_scored.append(list(df.iloc[i, :]))
    
#Test if Payload Data is correct
array_of_values_to_be_scored[1]

In [ ]:
import urllib3, requests, json

import sys,os,os.path

from watson_machine_learning_client import WatsonMachineLearningAPIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.0.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
# set space ID
space_uid = os.environ['SPACE_ID']
client.set.default_space(space_uid)

In [ ]:
print(client.deployments.list())

In [ ]:
# Select the model to evaluate (select from result of the above cell)
model_guid = "<replace with Model GUID"

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": payload_array_of_input_fields.tolist(), "values": array_of_values_to_be_scored}]}

job_details = client.deployments.score(model_guid,
                                        payload_scoring)
print("Scoring response")
print(job_details)

In [ ]:
predictions=[]
for x in range(len(df)):
    predictions.append(job_details['predictions'][0]['values'][x][0])
results_df["AutoAIPreds"] = predictions
results_df.head()

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, f1_score
print(results_df.AutoAIPreds.value_counts())
print(results_df.default.value_counts())
print(classification_report(results_df.default, results_df.AutoAIPreds))
print('ROC AUC: {:.2f}\n'.format(roc_auc_score(results_df.default, results_df.AutoAIPreds)))
print('F1 Score: {:.4f}\n'.format(f1_score(results_df.default, results_df.AutoAIPreds)))